In [7]:
import pandas as pd
from streamlit.testing.v1.element_tree import Dataframe
from torch.nn.functional import embedding

from serving.api import PROD_DATA_PATH

ref_path = "/Users/maximemoreau/Documents/ML/ML_deployement/data/ref_data.csv"
ref = pd.read_csv(ref_path)
prod_path = "/Users/maximemoreau/Documents/ML/ML_deployement/data/prod_data.csv"
prod = pd.read_csv(prod_path)

In [33]:
import joblib
from torchvision import datasets, transforms
import pandas as pd
from datetime import datetime
import os
from PIL import Image
import torch

CLASS_NAMES = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck"
}

# Charger le modèle et le scaler
model = joblib.load("/Users/maximemoreau/Documents/ML/ML_deployement/artifacts/model.pkl")

# Transformation pour les images (comme pour ResNet)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [34]:
prod.head()

,0,1,2,3,4,5,6,7,8,9,...,991,992,993,994,995,996,997,998,999,label
0,2024-11-28T03:18:35.387855,"[[0.15612497925758362, -0.003191523253917694, ...",3,cat,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-11-28T03:18:35.967468,"[[0.15612497925758362, -0.003191523253917694, ...",3,cat,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-11-28T03:18:36.435191,"[[0.15612497925758362, -0.003191523253917694, ...",3,cat,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-11-28T03:30:08.286354,"[[0.15612497925758362, -0.003191523253917694, ...",3,cat,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-11-28T03:30:41.510587,"[[0.15612497925758362, -0.003191523253917694, ...",3,cat,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
ref.head()

,0,1,2,3,4,5,6,7,8,9,...,991,992,993,994,995,996,997,998,999,label
0,-0.226760,0.069927,-0.549122,-0.880585,-0.396384,0.033055,-0.109720,-0.748466,0.034720,-0.334325,...,0.025356,-0.042504,1.020076,0.187329,1.068738,1.399755,0.197860,-0.375852,-0.125966,6.0
1,0.349531,0.020488,-0.505710,-0.462156,-0.476915,-0.006576,0.364920,-0.222701,-0.164957,-0.424240,...,0.036171,-0.076903,-0.104565,-0.135202,0.133758,0.069285,-0.107449,-0.315290,-0.282895,9.0
2,0.576189,0.002281,-1.256993,-0.775919,-0.489058,0.593333,0.055468,-0.510053,-0.632301,0.069436,...,-0.541868,-0.218030,-0.128817,0.551386,-0.178715,-0.248486,-0.287657,0.017741,-0.697421,9.0
3,0.658060,-0.037739,-0.624752,-0.798831,-0.798896,1.564715,-0.837681,-0.674321,-0.813789,0.799157,...,-0.682274,-0.238777,0.649445,-0.007221,1.056059,1.311593,-0.189407,-0.478287,-0.239074,4.0
4,0.119220,0.067307,-0.553441,0.016517,-0.488561,0.063167,0.094315,-0.862182,-0.141679,-0.118472,...,-0.182099,-0.262609,-0.270467,-0.447706,0.200527,-0.451478,-0.170289,-0.567564,-0.032854,1.0


In [68]:
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

image = Image.open("/Users/maximemoreau/Downloads/iStock-1332501286-scaled.jpg").convert("RGB")
image_tensor = transform(image).unsqueeze(0)  # Préparer l'image

from torchvision.models import resnet50, ResNet50_Weights
weights = ResNet50_Weights.DEFAULT
resnet = resnet50(weights=weights).eval()

with torch.no_grad():
    embedding = resnet(image_tensor).numpy()

    # Prédire la classe
predicted_class_id = model.predict(embedding)[0]
predicted_class_name = CLASS_NAMES[predicted_class_id]

*/    # Sauvegarder la prédiction dans prod_data.csv
data_to_save = {
    "timestamp": [datetime.now().isoformat()],
    "embedding": [embedding.tolist()],
    "predicted_class_id": [int(predicted_class_id)],
    "predicted_class_name": [predicted_class_name]
}
*/

SyntaxError: invalid syntax (1542071908.py, line 23)

In [67]:
df = pd.DataFrame(embedding,columns=[i for i in range(1000)])
df['label'] = predicted_class_id
PROD_DATA_PATH ="test.csv"
if not os.path.exists(PROD_DATA_PATH):  # Créer le fichier s'il n'existe pas
    df.to_csv(PROD_DATA_PATH, index=False)
else:
    df.to_csv(PROD_DATA_PATH, mode="a", header=False, index=False)

In [59]:
a = pd.DataFrame(embedding)

In [73]:
size(embedding)

NameError: name 'size' is not defined